In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_244282/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 13:23:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/11 13:23:52 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [8]:


data = {
    'player_id': [1, 1, 2, 3, 3],
    'device_id': [2, 2, 3, 1, 4],
    'event_date': ['2016-03-01', '2016-05-02', '2017-06-25', '2016-03-02', '2018-07-03'],
    'games_played': [5, 6, 1, 0, 5]
}

activity_df = pd.DataFrame(data)
activity_df['event_date'] = pd.to_datetime(activity_df['event_date'])

print("Activity table:")
print(activity_df)

# Now, let's find the latest device for each player
result_df = activity_df.sort_values(
    by='event_date', ascending=False).drop_duplicates(subset='player_id')

print("\nResult table:")
print(result_df[['player_id', 'device_id']].reset_index(drop=True))


df_person = spark.createDataFrame(activity_df)
df_person.createOrReplaceTempView("Activity")


data = {
    'player_id': [1, 1, 1, 2, 2, 3, 3, 3],
    'device_id': [2, 2, 3, 3, 4, 1, 4, 5],
    'event_date': ['2016-03-01', '2016-05-02', '2016-06-01', '2017-06-25', '2018-01-15', '2016-03-02', '2018-07-03', '2018-07-04'],
    'games_played': [5, 6, 0, 1, 3, 0, 5, 2]
}

activity_df = pd.DataFrame(data)
activity_df['event_date'] = pd.to_datetime(activity_df['event_date'])

print("Activity table:")
print(activity_df)

# Now, let's find the latest device for each player
result_df = activity_df.sort_values(
    by='event_date', ascending=False).drop_duplicates(subset='player_id')

print("\nResult table:")
print(result_df[['player_id', 'device_id']].reset_index(drop=True))


df_person = spark.createDataFrame(activity_df)
df_person.createOrReplaceTempView("Activity_1")

Activity table:
   player_id  device_id event_date  games_played
0          1          2 2016-03-01             5
1          1          2 2016-05-02             6
2          2          3 2017-06-25             1
3          3          1 2016-03-02             0
4          3          4 2018-07-03             5

Result table:
   player_id  device_id
0          3          4
1          2          3
2          1          2
Activity table:
   player_id  device_id event_date  games_played
0          1          2 2016-03-01             5
1          1          2 2016-05-02             6
2          1          3 2016-06-01             0
3          2          3 2017-06-25             1
4          2          4 2018-01-15             3
5          3          1 2016-03-02             0
6          3          4 2018-07-03             5
7          3          5 2018-07-04             2

Result table:
   player_id  device_id
0          3          5
1          2          4
2          1          3


In [13]:
query = """
	select
		player_id,
		device_id
 	from (select 
		a.player_id,
		a.device_id,
		a.event_date,
		row_number() over (partition by player_id order by event_date asc) as rank_date
	from Activity_1 a ) t
	where rank_date = 1
	

"""

In [14]:
spark.sql(query).show()

+---------+---------+
|player_id|device_id|
+---------+---------+
|        1|        2|
|        2|        3|
|        3|        1|
+---------+---------+

